# MNIST


In this project we will be using The MNISt dataset, which is a set of 70,000 small images of digits handwritten by high  school  students  and  employees  of  the  US  Cen‐sus Bureau. Each image is labeled with the digit it represents. This set has been stud‐ied  so  much  that  it  is  often  called  the  “hello  world”  of  Machine  Learning.
</br>
Scikit-Learn provides many helper functions to download popular datasets. MNIST isone of them. The following code fetches the MNIST dataset.

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
mnist.keys()